# 4.5. Weight Decay
 COLAB [MXNET]Open the notebook in Colab
 SAGEMAKER STUDIO LABOpen the notebook in SageMaker Studio Lab
Now that we have characterized the problem of overfitting, we can introduce some standard techniques for regularizing models. Recall that we can always mitigate overfitting by going out and collecting more training data. That can be costly, time consuming, or entirely out of our control, making it impossible in the short run. For now, we can assume that we already have as much high-quality data as our resources permit and focus on regularization techniques.

Recall that in our polynomial regression example (Section 4.4) we could limit our model’s capacity simply by tweaking the degree of the fitted polynomial. Indeed, limiting the number of features is a popular technique to mitigate overfitting. However, simply tossing aside features can be too blunt an instrument for the job. Sticking with the polynomial regression example, consider what might happen with high-dimensional inputs. The natural extensions of polynomials to multivariate data are called monomials, which are simply products of powers of variables. The degree of a monomial is the sum of the powers. For example, 
, and 
 are both monomials of degree 3.

Note that the number of terms with degree 
 blows up rapidly as 
 grows larger. Given 
 variables, the number of monomials of degree 
 (i.e., 
 multichoose 
) is 
. Even small changes in degree, say from 
 to 
, dramatically increase the complexity of our model. Thus we often need a more fine-grained tool for adjusting function complexity.

## 4.5.1. Norms and Weight Decay
We have described both the 
 norm and the 
 norm, which are special cases of the more general 
 norm in Section 2.3.10. Weight decay (commonly called 
 regularization), might be the most widely-used technique for regularizing parametric machine learning models. The technique is motivated by the basic intuition that among all functions 
, the function 
 (assigning the value 
 to all inputs) is in some sense the simplest, and that we can measure the complexity of a function by its distance from zero. But how precisely should we measure the distance between a function and zero? There is no single right answer. In fact, entire branches of mathematics, including parts of functional analysis and the theory of Banach spaces, are devoted to answering this issue.

One simple interpretation might be to measure the complexity of a linear function 
 by some norm of its weight vector, e.g., 
. The most common method for ensuring a small weight vector is to add its norm as a penalty term to the problem of minimizing the loss. Thus we replace our original objective, minimizing the prediction loss on the training labels, with new objective, minimizing the sum of the prediction loss and the penalty term. Now, if our weight vector grows too large, our learning algorithm might focus on minimizing the weight norm 
 vs. minimizing the training error. That is exactly what we want. To illustrate things in code, let us revive our previous example from Section 3.1 for linear regression. There, our loss was given by

(4.5.1)
 
 
 
Recall that 
 are the features, 
 are labels for all data examples 
, and 
 are the weight and bias parameters, respectively. To penalize the size of the weight vector, we must somehow add 
 to the loss function, but how should the model trade off the standard loss for this new additive penalty? In practice, we characterize this tradeoff via the regularization constant 
, a non-negative hyperparameter that we fit using validation data:

(4.5.2)
 
For 
, we recover our original loss function. For 
, we restrict the size of 
. We divide by 
 by convention: when we take the derivative of a quadratic function, the 
 and 
 cancel out, ensuring that the expression for the update looks nice and simple. The astute reader might wonder why we work with the squared norm and not the standard norm (i.e., the Euclidean distance). We do this for computational convenience. By squaring the 
 norm, we remove the square root, leaving the sum of squares of each component of the weight vector. This makes the derivative of the penalty easy to compute: the sum of derivatives equals the derivative of the sum.

Moreover, you might ask why we work with the 
 norm in the first place and not, say, the 
 norm. In fact, other choices are valid and popular throughout statistics. While 
-regularized linear models constitute the classic ridge regression algorithm, 
-regularized linear regression is a similarly fundamental model in statistics, which is popularly known as lasso regression.

One reason to work with the 
 norm is that it places an outsize penalty on large components of the weight vector. This biases our learning algorithm towards models that distribute weight evenly across a larger number of features. In practice, this might make them more robust to measurement error in a single variable. By contrast, 
 penalties lead to models that concentrate weights on a small set of features by clearing the other weights to zero. This is called feature selection, which may be desirable for other reasons.

Using the same notation in (3.1.10), the minibatch stochastic gradient descent updates for 
-regularized regression follow:

(4.5.3)
 
 
 
As before, we update 
 based on the amount by which our estimate differs from the observation. However, we also shrink the size of 
 towards zero. That is why the method is sometimes called “weight decay”: given the penalty term alone, our optimization algorithm decays the weight at each step of training. In contrast to feature selection, weight decay offers us a continuous mechanism for adjusting the complexity of a function. Smaller values of 
 correspond to less constrained 
, whereas larger values of 
 constrain 
 more considerably.

Whether we include a corresponding bias penalty 
 can vary across implementations, and may vary across layers of a neural network. Often, we do not regularize the bias term of a network’s output layer.

## 4.5.2. High-Dimensional Linear Regression
We can illustrate the benefits of weight decay through a simple synthetic example.

In [1]:
use strict;
use warnings;
use Data::Dump qw(dump);
use AI::MXNet qw(mx);
use AI::MXNet::Gluon qw(gluon);
use List::Util qw(min max shuffle);
use d2l;
use d2l::Accumulator;
use d2l::Animator;
IPerl->load_plugin('Chart::Plotly');

First, we generate some data as before

(4.5.4)
 
We choose our label to be a linear function of our inputs, corrupted by Gaussian noise with zero mean and standard deviation 0.01. To make the effects of overfitting pronounced, we can increase the dimensionality of our problem to 
 and work with a small training set containing only 20 examples.

In [2]:
sub load_array{
    my ($features, $labels, $batch_size, $shuffle, $seed) = @_;
    srand ($seed) if defined $seed;
    my $num_samples = $features->len;
    my @indices = $shuffle ? shuffle (0 .. $num_samples - 1) : (0 .. $num_samples - 1);
    my ($index, @batch_indices) = 0;
    $features = $features->asarray;
    $labels = $labels->asarray;
    return sub {
        if (defined $_[0] && $_[0] == 0) {
        @indices = shuffle @indices if $shuffle;
        $index = 0;
        return 1;
     }
     return undef if ($index >= $num_samples);
     @batch_indices = @indices[$index .. min($index + $batch_size, $num_samples) - 1];
     $index += $batch_size;
     return {data => mx->nd->array([@$features [@batch_indices]]),
             label => mx->nd->array([@$labels [@batch_indices]])};
  };
}


In [3]:
my ($n_train, $n_test, $batch_size) = (20, 100, 5);
our $num_inputs = 200;
my ($true_w, $true_b) = (mx->nd->ones([$num_inputs, 1]) * 0.01, 0.05);
our ($train_data, $labels) = d2l->synthetic_data($true_w, $true_b, $n_train);
our $train_iter = load_array($train_data, $labels, $batch_size, 0);
my ($test_data, $test_labels)  = d2l->synthetic_data($true_w, $true_b, $n_test);
our $test_iter = load_array($test_data, $test_labels, $batch_size, is_train => 1 );

CODE(0xb124c08)

## 4.5.3. Implementation from Scratch
In the following, we will implement weight decay from scratch, simply by adding the squared 
 penalty to the original target function.

### 4.5.3.1. Initializing Model Parameters
First, we will define a function to randomly initialize our model parameters.

In [4]:
sub init_params{
    our $num_inputs;
    my $w = mx->nd->random->normal(scale => 1, shape => [$num_inputs, 1]);
    my $b = mx->nd->zeros([1]);
    $w->attach_grad();
    $b->attach_grad();
    return ($w, $b);
}

In [5]:
init_params();

<AI::MXNet::NDArray 200x1 @cpu(0)><AI::MXNet::NDArray 1 @cpu(0)>

### 4.5.3.2. Defining  Norm Penalty
Perhaps the most convenient way to implement this penalty is to square all terms in place and sum them up.

In [6]:
sub l2_penalty{
    my $w = shift;
    return (($w->sum()) / 2);
}

### 4.5.3.3. Defining the Training Loop
The following code fits a model on the training set and evaluates it on the test set. The linear network and the squared loss have not changed since Section 3, so we will just import them via d2l.linreg and d2l.squared_loss. The only change here is that our loss now includes the penalty term.

In [12]:
sub train{
    my $lambd = @_;
    my ($w, $b) = init_params();
    my $net = \&{'linreg'};
    my $loss = \&{'squared_loss'};
    my ($num_epochs, $lr) = (100, 0.003);
  #  my $animator = Animator();
    our ($train_iter, $test_iter);
    our ($train_data, $labels);
    for my $epoch (1 .. $num_epochs) {
        while (my $minibatch = $train_iter->()){
            my $X = $minibatch->{data};
            my $y = $minibatch->{label};
            my $l;
            autograd->record(sub {
                $l = (($loss->($net->($X), $y))+ $lambd * l2_penalty($w));
             });
            $l->backward();
           # if (($epoch + 1) % 5 == 0) {
             #   $animator->add(($epoch + 1), (d2l->evaluate_loss($net, $train_iter, $loss),
             #                                   d2l->evaluate_loss($net, $test_iter, $loss)));
            #    }
            
            }
        }
        my $traces->[0] = Chart::Plotly::Trace::Scatter->new(name => "True Regression Line",
                                                   x => $train_data->asarray,
                                                   y => $labels->asarray,
                                                   mode => "lines",
                                                   line => {color => 'blue',
                                                   width => 3});
    print('L2 norm of w:', dump mx->nd->norm($w)->asarray);
}


Warning: Subroutine train redefined at reply input line 1.


### 4.5.3.4. Training without Regularization
We now run this code with lambd = 0, disabling weight decay. Note that we overfit badly, decreasing the training error but not the test error—a textbook case of overfitting.

In [13]:
train(0);

L2 norm of w:[14.2511596679688]

1

### 4.5.3.5. Using Weight Decay
Below, we run with substantial weight decay. Note that the training error increases but the test error decreases. This is precisely the effect we expect from regularization.

In [14]:
train(3);  

L2 norm of w:[14.513542175293]

1

## 4.5.4. Concise Implementation
Because weight decay is ubiquitous in neural network optimization, the deep learning framework makes it especially convenient, integrating weight decay into the optimization algorithm itself for easy use in combination with any loss function. Moreover, this integration serves a computational benefit, allowing implementation tricks to add weight decay to the algorithm, without any additional computational overhead. Since the weight decay portion of the update depends only on the current value of each parameter, the optimizer must touch each parameter once anyway.

In [10]:
sub train_concise{
    my $wd = shift;
    my $net = nn->Sequential();
    $net->add(nn->Dense(1));
    $net->initialize(mx->init->Normal(sigma => 1));
    my $loss = gluon->loss->L2Loss();
    my ($num_epochs, $lr) = (100, 0.003);
    my $trainer = gluon->Trainer($net->collect_params(),
                                optimizer => 'sgd',
                                optimizer_params => { learning_rate => $lr, wd=> $wd});
    # The bias parameter has not decayed. Bias names generally end with "bias"
    $net->collect_params('.*bias')->setattr('wd_mult', 0);
    #y $animator = Animator(xlabel => 'epochs', ylabel => 'loss', yscale => 'log',
     #                      xlim => [5, $num_epochs], legend => ['train', 'test']);
    our ($train_iter, $test_iter);
    our ($train_data, $labels);
    
    for my $epoch (1 .. $num_epochs) {
        while (my $minibatch = $train_iter->()){
            my $X = $minibatch->{data};
            my $y = $minibatch->{label};
            my $l;
            autograd->record(sub {
                $l = $loss->($net->($X), $y);
             });
            $l->backward();
           # if (($epoch + 1) % 5 == 0) {
             #   $animator->add(($epoch + 1), (d2l->evaluate_loss($net, $train_iter, $loss),
             #                                   d2l->evaluate_loss($net, $test_iter, $loss)));
            #    }
            
            }
        }
    my $traces->[0] = Chart::Plotly::Trace::Scatter->new(name => "True Regression Line",
                                                   x => $train_data->asarray,
                                                   y => $labels->asarray,
                                                   mode => "lines",
                                                   line => {color => 'blue',
                                                   width => 3});

    print('L2 norm of w:', mx->nd->norm($net->weight->data()->at(0)));  
}

The plots look identical to those when we implemented weight decay from scratch. However, they run appreciably faster and are easier to implement, a benefit that will become more pronounced for larger problems.

In [11]:
train_concise(0);

Error: Can't call method "data" on an undefined value at reply input line 41.


So far, we only touched upon one notion of what constitutes a simple linear function. Moreover, what constitutes a simple nonlinear function can be an even more complex question. For instance, reproducing kernel Hilbert space (RKHS) allows one to apply tools introduced for linear functions in a nonlinear context. Unfortunately, RKHS-based algorithms tend to scale poorly to large, high-dimensional data. In this book we will default to the simple heuristic of applying weight decay on all layers of a deep network.

## 4.5.5. Summary
Regularization is a common method for dealing with overfitting. It adds a penalty term to the loss function on the training set to reduce the complexity of the learned model.

One particular choice for keeping the model simple is weight decay using an  penalty. This leads to weight decay in the update steps of the learning algorithm.

The weight decay functionality is provided in optimizers from deep learning frameworks.

Different sets of parameters can have different update behaviors within the same training loop.